## Import der relevanten packages

In [2]:
# Importieren der nötigen Librarys
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

## Einladen der Datasets

In [3]:
# Einlesen des Trainingdatensatzes
url_train = "https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/train_dataf_knn.csv"
train_data = pd.read_csv(url_train)

# Prpüfen des eigeladenen Datensatzes
print(train_data.shape)
print(train_data.isnull().sum()) # shows number of missing values -> (NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike)

print(train_data)

# Einlesen des Validierungsdatensatz

url_validation = "https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/validation_dataf_knn.csv"
validation_data = pd.read_csv(url_validation)

# Prpüfen des eigeladenen Datensatzes
print(validation_data.shape)
print(validation_data.isnull().sum()) # shows number of missing values -> (NaN in numeric arrays, None or NaN in object arrays, NaT in datetimelike)

print(validation_data)




(7523, 20)
Datum                    0
KielerWoche              0
Warengruppe              0
Umsatz                   0
Bewoelkung               0
Temperatur               0
Windgeschwindigkeit      0
Wettercode               0
FerienSH                 0
Feiertag                 0
Umschlag                 0
Weihnachtsmarkt          0
Verbraucherpreisindex    0
Regen                    0
Schnee                   0
Wochentag_MDMDFSS        0
Wochenende               0
Jahreszeit_FSHW          0
Temperatur_Kategorie     0
THW_heimspiel            0
dtype: int64
           Datum  KielerWoche  Warengruppe      Umsatz  Bewoelkung  \
0     2013-07-01          0.0            1  148.828353           6   
1     2013-07-01          0.0            2  535.856285           6   
2     2013-07-01          0.0            3  201.198426           6   
3     2013-07-01          0.0            4   65.890169           6   
4     2013-07-01          0.0            5  317.475875           6   
...          ...

## Entfernen der Datumsvariable für die schrittweise Selektion der Features mittels Algorithmus

In [4]:
# Entfernen der 'Datum'-Spalte aus allen Datensätzen
train_data = train_data.drop(columns=['Datum'])
validation_data = validation_data.drop(columns=['Datum'])


## Auswahl der Features mit einfacher linearer Regression

### Schrittweise Selektion der Features mittels Algorithmus basierend auf den Trainingsdaten

In [5]:
# Schrittweise Auswahl der Merkmale basierend auf den Trainingsdaten
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# Zielvariable und Merkmale für Trainingsdaten
X_train = train_data.drop(columns=['Umsatz'])
y_train = train_data['Umsatz']

# Durchführen der schrittweisen Merkmalsauswahl
selected_features = stepwise_selection(X_train, y_train)

print (selected_features)


Add  Temperatur                     with p-value 2.47916e-80
Add  Wochenende                     with p-value 3.03835e-45
Add  FerienSH                       with p-value 2.19206e-33
Add  Feiertag                       with p-value 3.7228e-13
Add  Verbraucherpreisindex          with p-value 4.29835e-09
Add  Warengruppe                    with p-value 5.06715e-06
Add  Weihnachtsmarkt                with p-value 5.84196e-05
Add  KielerWoche                    with p-value 0.00181527
['Temperatur', 'Wochenende', 'FerienSH', 'Feiertag', 'Verbraucherpreisindex', 'Warengruppe', 'Weihnachtsmarkt', 'KielerWoche']


## Aufstellen des Modells mit den gewählten Features 

In [6]:
# Training des Modells mit den ausgewählten Merkmalen mit Hilfe von linearer Regression
model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()

print (model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     110.8
Date:                Mon, 22 Jul 2024   Prob (F-statistic):          1.22e-175
Time:                        06:27:58   Log-Likelihood:                -47826.
No. Observations:                7523   AIC:                         9.567e+04
Df Residuals:                    7514   BIC:                         9.573e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1236.94

## Prüfen der Vorhersagen mit den Validierungsdaten

In [7]:
# Zielvariable und Merkmale für Validierungsdaten
X_val = validation_data.drop(columns=['Umsatz'])
y_val = validation_data['Umsatz']

# Vorhersagen auf den Validierungsdaten
y_val_pred = model.predict(sm.add_constant(X_val[selected_features]))

# Validierungsmetriken berechnen
r2_val = r2_score(y_val, y_val_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

print(f'Validierungs-R²: {r2_val:.2f}')
print(f'Validierungs-MSE: {mse_val:.2f}')
print(f'Validierungs-MAE: {mae_val:.2f}')
print(f"MAPE: {mape_val}%")


Validierungs-R²: 0.10
Validierungs-MSE: 15101.73
Validierungs-MAE: 94.97
MAPE: 65.08796225813174%


# Feature Selection mit Hilfe des Random Forest Regressors  als Baseline Modell 

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm


In [8]:
# Daten vorbereiten
X_train = train_data.drop(columns=['Umsatz'])
y_train = train_data['Umsatz']
X_val = validation_data.drop(columns=['Umsatz'])
y_val = validation_data['Umsatz']


## Mit Hilfe des Random Forest Modells soll der beste Fit eines Vorhersage-Modells ermittelt werden. Anhand der Feature Importance sollen nur die 10 wichtigsten Features mit in das Modell einfließen

In [9]:
# Random Forest Modell für Feature-Importance
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Feature Importances extrahieren und sortieren
importances = rf.feature_importances_
indices = importances.argsort()[::-1]

# Anzahl der zu wählenden Features
n_features = 10
selected_indices = indices[:n_features]
selected_features = X_train.columns[selected_indices]

# Anzeigen der ausgewählten Features
print("Ausgewählte Features:", selected_features.tolist())

# Auswahl der wichtigsten Features
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]


# Hyperparameter-Tuning mit GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 30, None]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# Bestes Modell
best_rf = grid_search.best_estimator_



Ausgewählte Features: ['Warengruppe', 'Temperatur', 'Wochentag_MDMDFSS', 'Verbraucherpreisindex', 'FerienSH', 'Wochenende', 'Feiertag', 'Jahreszeit_FSHW', 'Windgeschwindigkeit', 'Wettercode']


# Bewertung der errechneten Features aus dem besten Random Forest Modells anhand der Validierungsdaten

In [10]:

# Vorhersagen und Bewertung auf den Validierungsdaten
y_val_pred = best_rf.predict(X_val_selected)
r2_val = r2_score(y_val, y_val_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

# Berechnung des adjusted R²
n = X_val_selected.shape[0]
p = X_val_selected.shape[1]
adjusted_r2_val = 1 - (1 - r2_val) * (n - 1) / (n - p - 1)

print(f'Validation-R²: {r2_val:.2f}')
print(f'Validation-MSE: {mse_val:.2f}')
print(f'Validation-MAE: {mae_val:.2f}')
print(f'Validation-MAPE: {mape_val:.2f}%')
print(f'Validation-adjusted R²: {adjusted_r2_val:.2f}')


Validation-R²: 0.83
Validation-MSE: 2934.48
Validation-MAE: 35.48
Validation-MAPE: 24.86%
Validation-adjusted R²: 0.82


### Test verschiedener Kombinationen der errechneten Features in linearen Modellen

In [12]:
mod = smf.ols('Umsatz ~ C(Warengruppe) + C(Temperatur) + C(Wochentag_MDMDFSS) + C(Verbraucherpreisindex) + C(FerienSH) +C(Wochenende) + C(Feiertag)+ C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)+ C(Wettercode)', data=train_data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     35.17
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:55:55   Log-Likelihood:                -42411.
No. Observations:                7523   AIC:                         8.626e+04
Df Residuals:                    6803   BIC:                         9.125e+04
Df Model:                         719                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [13]:
mod2 = smf.ols('Umsatz ~ C(Warengruppe)*C(Temperatur) + C(Wochentag_MDMDFSS) + C(Verbraucherpreisindex) + C(FerienSH) + C(Wochenende)+ C(Feiertag)+ C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)+ C(Wettercode)', data=train_data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     35.17
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:57:28   Log-Likelihood:                -42411.
No. Observations:                7523   AIC:                         8.626e+04
Df Residuals:                    6803   BIC:                         9.125e+04
Df Model:                         719                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [14]:
mod3 = smf.ols('Umsatz ~ C(Warengruppe)*C(Temperatur) + C(Warengruppe)*C(Jahreszeit_FSHW)+ C(Wochentag_MDMDFSS) + C(Verbraucherpreisindex) + C(FerienSH) + C(Wochenende)+ C(Feiertag)+ C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)+ C(Wettercode)', data=train_data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     35.17
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:58:43   Log-Likelihood:                -42411.
No. Observations:                7523   AIC:                         8.626e+04
Df Residuals:                    6803   BIC:                         9.125e+04
Df Model:                         719                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [15]:
mod4 = smf.ols('Umsatz ~ C(Warengruppe)*C(Temperatur) + C(Temperatur)*C(Jahreszeit_FSHW)+ C(Wochentag_MDMDFSS) + C(Verbraucherpreisindex) + C(FerienSH) + C(Wochenende)+ C(Feiertag)+ C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)+ C(Wettercode)', data=train_data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     35.17
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        07:00:59   Log-Likelihood:                -42411.
No. Observations:                7523   AIC:                         8.626e+04
Df Residuals:                    6803   BIC:                         9.125e+04
Df Model:                         719                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

: 

In [10]:
mod5 = smf.ols('Umsatz ~ C(Warengruppe)*C(Temperatur) + C(Warengruppe)* C(Wettercode) + C(Wochentag_MDMDFSS) + C(Verbraucherpreisindex) + C(FerienSH) + C(Wochenende)+ C(Feiertag)+ C(Jahreszeit_FSHW) + C(Windgeschwindigkeit)+ C(Wettercode)', data=train_data).fit()
print(mod.summary())

NameError: name 'mod' is not defined